In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import utils

from einops import rearrange, reduce, repeat
from sklearn.metrics import mean_squared_error

In [ ]:
import importlib
importlib.reload(utils)


In [13]:
nTrial = 20
wSize = 60
nReachs = range(1, 20, 1)

# xy_nReach: [nReach][nTrial]
xy_nReach = []
for nReach in nReachs:
    xy, _ = utils.SynthData.genReachingSeq_trial(nTrial, nReach=nReach)
    xy = [utils.DataProcessing.rollingWindow(xy_, wSize) for xy_ in xy]
    xy_nReach.append(xy)


In [8]:
subjs = utils.ExpInfo.getSubjIDs()
task = utils.ExpInfo.taskName[2]


In [21]:
def reconstruct_mse(subj, X):
    H, Y = utils.Model.quick_forward(subj, X)
    mse = []
    for iTrial in range(nTrial):
        x = X[iTrial]
        y = Y[iTrial]
        x = rearrange(x, 'b t f -> (b t) f')
        y = rearrange(y, 'b t f -> (b t) f')
        mse.append(mean_squared_error(x, y))
    return np.mean(mse)

def reconstruct_mse_subj(subj):
    mse = []
    for iReach in range(len(nReachs)):
        mse_ = reconstruct_mse(subjs[0], xy_nReach[iReach])
        mse.append(mse_)        
    return mse

mse = utils.GroupOperation.map(reconstruct_mse_subj, subjs)

Group loop |████████████████████████████████████████| 50/50 [100%] in 21:51.3 (0.04/s)                                  


In [23]:
import pickle
savepath = utils.path_data / 'ana_reaching_measuring_reaching_ability'
fname = savepath / 'mse.pickle'
with open(fname, "wb") as f:
    pickle.dump(mse, f)

